In [1]:
#일반 라이브러리
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#API를 사용하기 위한 import
# from geoband.API import *
# import folium
# import json
# import geopandas as gpd

# 전처리 및 operator
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


# model
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV



In [2]:
correct = pd.read_csv("./correct_answer.csv")
correct = correct.iloc[:,1:]

q3 = pd.read_csv("./q3.csv")
q3 = q3.iloc[:,1:]

q5 = pd.read_csv("./q5.csv")
q5 = q5.iloc[:,1:]

In [3]:
answer = pd.concat([correct.iloc[:,[0,1]], q3, correct.iloc[:,3], q5], axis = 1)
Basic = pd.read_csv("./Basic.csv")

answers = pd.concat([Basic,answer], axis = 1)
answers = answers.drop(["year","상반기"], axis = 1)

answers = answers.groupby("jur_stn").mean().reset_index()
answers.columns = ["jur_stn","q1","q2","q3","q4","q5"]
answers.head()

,jur_stn,q1,q2,q3,q4,q5
0,마산동부경찰서,75.655658,77.697778,63.641550,69.337500,69.027147
1,마산중부경찰서,78.186459,79.765556,66.189649,71.341667,71.125655
2,서울강남경찰서,80.225870,81.517778,62.443416,72.358333,72.465293
3,서울강동경찰서,77.529867,79.707778,70.742150,70.733333,74.252083
4,서울강북경찰서,74.745683,76.415556,72.803059,67.079167,73.339542


In [4]:
answers["범죄 안전도"] = (answers["q1"] + answers["q2"]) /2
answers["분야별 안전도"] = answers["범죄 안전도"]*0.343 + answers["q3"]*0.305 + answers["q4"]*0.352
answers["종합체감안전도_점수"] = answers["q5"]*0.3 + answers["분야별 안전도"]*0.7

answers = answers[["jur_stn","종합체감안전도_점수", "분야별 안전도", "범죄 안전도", "q1","q2","q3","q4","q5"]]
answers.head()

,jur_stn,종합체감안전도_점수,분야별 안전도,범죄 안전도,q1,q2,q3,q4,q5
0,마산동부경찰서,69.790455,70.117587,76.676718,75.655658,77.697778,63.641550,69.337500,69.027147
1,마산중부경찰서,72.009913,72.388880,78.976007,78.186459,79.765556,66.189649,71.341667,71.125655
2,서울강남경찰서,72.317676,72.254411,80.871824,80.225870,81.517778,62.443416,72.358333,72.465293
3,서울강동경찰서,73.684146,73.440745,78.618822,77.529867,79.707778,70.742150,70.733333,74.252083
4,서울강북경찰서,72.220529,71.740952,75.580619,74.745683,76.415556,72.803059,67.079167,73.339542


In [5]:
submit = pd.read_csv("./25.결과제출양식.csv")
police = submit["경찰서명"]

In [6]:
for i in range(len(police)):
    temp = answers[answers["jur_stn"] == police[i].strip()].iloc[:,1:]
    submit.iloc[i,2:] = temp.iloc[0,:].tolist()

In [7]:
submit.head()

,순번,경찰서명,종합체감안전도_점수 \n(전반적안전도*0.3) + (분야별안전도*0.7),분야별 안전도\n(범죄안전도*0.343) + \n(교통사고안전도*0.305)\n + (법질서준수도*0.352),범죄 안전도\n(절도폭력안전도+강도살인안전도)/2,문항1. 절도 폭력안전도\n (문항1 점수 * 10),문항2. 강도 살인 안전도\n(문항2 점수 * 10),문항3. 교통사고 안전도\n(문항3 점수*10),문항4. 법질서 준수도\n(문항 4점수 * 10),문항5. 전반적 안전도\n(문항 5점수 * 10)
0,1,서울중부경찰서,73.001350,72.449134,79.414128,78.147146,80.681111,70.626301,67.241667,74.289854
1,2,서울종로경찰서,74.532938,74.475105,82.521536,81.377516,83.665556,69.403118,71.029167,74.667884
2,3,서울남대문경찰서,74.173040,73.512538,79.021346,77.838247,80.204444,74.309342,67.454167,75.714212
3,4,서울서대문경찰서,72.911824,72.271147,78.089893,77.805341,78.374444,69.666164,68.858333,74.406738
4,5,서울혜화경찰서,72.529781,71.418590,78.709187,78.000596,79.417778,71.079362,64.608333,75.122561


In [11]:
submit.to_csv("./submit3.csv", encoding="utf-8", index= False)